In [ ]:
import ee
import os
import geemap
import geemap.foliumap as geemap

# VISUALIZACIÓN DE INCENDIOS
En el siguiente panel se muestra en la ventana izquierda una imagen del 26-04-2018 con el efecto causado por el incendio en Indio Maíz, miestras que en el panel derecho se muestra una imagen del 25-01-2022 en el que se muestra la recuperación de la zona afectada. Puede arrastrar el botón central para establecer comparaciones del antes y despúes, también puede dar zoom a la imagen con el scroll del mause.

In [ ]:
Map = geemap.Map(center=(11.0, -83.7), zoom=11)
Map.add_basemap('OpenTopoMap')

before = ee.Image('COPERNICUS/S2/20180426T160509_20180426T160709_T16PHT').select(['B11', 'B8', 'B2'])
after = ee.Image('COPERNICUS/S2/20220125T160509_20220125T160505_T16PHT').select(['B11', 'B8', 'B2'])

left_layer = geemap.ee_tile_layer(before, {'min':1000, 'max': 3500}, 'Antes')
right_layer = geemap.ee_tile_layer(after, {'min': 2000, 'max': 4500}, 'Después')

Map.split_map(left_layer, right_layer)

Map

# VISUALIZACIÓN DE INUNDACIONES
En la siguiente interfaz podrá obsrvar cuales zonas sufrieron inundaciones a causa del huracán Eta e Iota en noviembre del 2021, porá observar en color negro las zonas donde hay agua permanente, mientras que las zonas inundadas se muestran en color rojo. en la esquina superior derecha, junto al icono de yave crece aparece el botón "Layers", aqui puede activar o desactivar la visualización de la imagen para observar si las zonas inundadas corresponden a zonas agrícolas.

In [ ]:
# DEFINE EL MAPA Y EL ZOOM
Map = geemap.Map(center = (13.6, -83.85), zoom = 10)
Map.add_basemap('HYBRID')

# DEFINE EL AREA DE INTERÉS
# AOI = ee.FeatureCollection('users/Guillermo-Ortega/Limites/Croquis_Nacional')

AOI = ee.FeatureCollection('users/Guillermo-Ortega/Limites/Departamentos')\
            .filter(ee.Filter.inList('CODIGO', [91]));

# AOI = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
#         .filter(ee.Filter.eq('country_na', 'Nicaragua'))

# ACCEDE A LA COLECCIÓN DE IMÁGENES
coleccion = ee.ImageCollection("COPERNICUS/S1_GRD")\
              .filter(ee.Filter.eq('instrumentMode', 'IW'))\
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
              .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
              .filterMetadata('resolution_meters', 'equals', 10)\
              .filterBounds(AOI).select ('VH');

# DEFINE EL ANTES Y EL DESPÉS
antes = coleccion.filterDate('2022-05-01', '2022-05-20')\
                .median()\
                .clip(AOI)\
                .focal_mean(30, 'square', 'meters')\
                .int();
despues = coleccion.filterDate('2022-06-01', '2022-06-30')\
                .median()\
                .clip(AOI)\
                .focal_mean(30, 'square', 'meters')\
                .int();

# CREAR EL MULTIBANDA Y MOSTRAR EN EL MAPA
RADAR_RGB = antes.addBands(despues).addBands(despues);

# CREAR MASCARAS DE AGUA Y MOSTRARLAS EN EL MAPA
aguaDespues = despues.updateMask(despues.lt(-20)).multiply(-1);
Map.addLayer(aguaDespues, {'palette': ['FF0000']}, 'Agua despues', True, 1);

aguaAntes = antes.updateMask(antes.lt(-20)).multiply(-1);
Map.addLayer(aguaAntes, {'palette': ['00FFFF']}, 'Agua antes', True, 1);

Map.addLayer(RADAR_RGB, {'min': -25, 'max': -12}, 'Imagen SAR Sentinel-1', False, 1);

Map.addLayer(AOI, {'color': '00000000', 'width': 1, 'lineType': 'solid', 'fillColor': '00000000'}, 'Límites')

# # ELABORAR UNA LEYENDA Y MOSTRARLA EN EL MAPA
# elemetos_leyenda = {'Inundación': 'FF0000', 'Agua permanente': '000000',}
# Map.add_legend(legend_title = 'Simbología', legend_dict = elemetos_leyenda, position = 'bottomright')

Map

In [ ]:
Map.publish(name= 'Áreas inundadas en la Costa Caribe Norte, junio 2022',
           description= 'Una visualización a través de RADAR de las zonas inundadas')